In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from peft import LoraConfig, get_peft_model, TaskType

In [2]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

In [3]:
lora_config = LoraConfig(
    r=8,  # Reduced rank
    lora_alpha=8,  # Lower scaling factor
    #target_modules=["q"],  # Update fewer modules (e.g., only query weights)
    lora_dropout=0.1,  # Increased dropout for better regularization
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM  # Task type remains the same
)

In [4]:
model_name='google/flan-t5-small'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [7]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


In [8]:
peft_model = get_peft_model(model, 
                            lora_config)

In [9]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 344064
all model parameters: 77305216
percentage of trainable model parameters: 0.45%


In [10]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

In [11]:
# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")
print(tokenized_datasets)

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


In [13]:
output_dir = f'./SumSmart-training'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_steps=1
    )
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
    )

In [14]:
peft_trainer.train()

Step,Training Loss
1,60.500000
2,55.000000
3,55.750000
4,51.500000
5,51.750000
6,48.750000
7,45.500000
8,46.500000
9,43.750000
10,43.250000


TrainOutput(global_step=16, training_loss=46.8125, metrics={'train_runtime': 6331.3059, 'train_samples_per_second': 0.02, 'train_steps_per_second': 0.003, 'total_flos': 23368433664000.0, 'train_loss': 46.8125, 'epoch': 1.0})

In [15]:
peft_model_path="./SumSmart-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./SumSmart-checkpoint-local/tokenizer_config.json',
 './SumSmart-checkpoint-local/special_tokens_map.json',
 './SumSmart-checkpoint-local/tokenizer.json')